In [5]:
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten, LSTM, Bidirectional
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 選擇數據處理模式：'N-R' 或 'R'
mode = 'N-R'  # 可以選擇 'N-R' 或 'R'

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2014test-A.txt',
    './dataset/train/twitter-2015train-A.txt'
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練 Word2Vec 模型
tokenized_sentences = [tweet.split() for tweet in dataset['tweet'].values]
word2vec_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=EMBEDDING_DIM,  # 詞向量維度為 25
                          window=5,                   # 最大跳躍距離設為 5
                          min_count=5,                # 最低頻次為 5
                          sg=0,                       # CBOW 模型 (sg=0)
                          workers=4)                  # 使用 4 個核心進行訓練

# 將詞向量進行標準化
def normalize_vector(vector):
    v_min, v_max = np.min(vector), np.max(vector)
    return (vector - v_min) / (v_max - v_min) if v_max > v_min else vector

# 構建嵌入矩陣
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[i] = normalize_vector(embedding_vector)

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

In [6]:
from keras.layers import Flatten

# 建立單一 CNN 模型並加入 Flatten 層
cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 1s 4ms/step - loss: 1.0086 - accuracy: 0.4596 - val_loss: 0.9834 - val_accuracy: 0.4938
Epoch 2/10
301/301 [==============================] - 1s 4ms/step - loss: 0.9870 - accuracy: 0.4763 - val_loss: 0.9759 - val_accuracy: 0.4925
Epoch 3/10
301/301 [==============================] - 1s 4ms/step - loss: 0.9778 - accuracy: 0.4913 - val_loss: 0.9701 - val_accuracy: 0.4929
Epoch 4/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9746 - accuracy: 0.4910 - val_loss: 0.9672 - val_accuracy: 0.4892
Epoch 5/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9732 - accuracy: 0.4940 - val_loss: 0.9718 - val_accuracy: 0.4938
Epoch 6/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9717 - accuracy: 0.4950 - val_loss: 0.9683 - val_accuracy: 0.4900
Epoch 7/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9694 - accuracy: 0.4955 - val_loss: 0.9695 - val_accuracy: 0.4821
Epoch 

In [7]:
# 單一 LSTM 模型 (Single LSTM)

from keras.layers import LSTM

# 建立單一 LSTM 模型
lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 19s 57ms/step - loss: 0.9946 - accuracy: 0.4661 - val_loss: 0.9753 - val_accuracy: 0.4904
Epoch 2/10
301/301 [==============================] - 17s 55ms/step - loss: 0.9801 - accuracy: 0.4888 - val_loss: 0.9668 - val_accuracy: 0.5175
Epoch 3/10
301/301 [==============================] - 17s 56ms/step - loss: 0.9757 - accuracy: 0.4914 - val_loss: 0.9576 - val_accuracy: 0.5158
Epoch 4/10
301/301 [==============================] - 17s 56ms/step - loss: 0.9716 - accuracy: 0.4999 - val_loss: 0.9702 - val_accuracy: 0.4913
Epoch 5/10
301/301 [==============================] - 17s 56ms/step - loss: 0.9755 - accuracy: 0.4952 - val_loss: 0.9641 - val_accuracy: 0.4904
Epoch 6/10
301/301 [==============================] - 17s 57ms/step - loss: 0.9726 - accuracy: 0.4964 - val_loss: 0.9601 - val_accuracy: 0.5141
Epoch 7/10
301/301 [==============================] - 17s 55ms/step - loss: 0.9699 - accuracy: 0.4995 - val_loss: 0.9571 - val_accuracy:

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
# CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 8s 20ms/step - loss: 1.0060 - accuracy: 0.4572 - val_loss: 0.9838 - val_accuracy: 0.5004
Epoch 2/10
301/301 [==============================] - 6s 19ms/step - loss: 0.9886 - accuracy: 0.4777 - val_loss: 0.9864 - val_accuracy: 0.4821
Epoch 3/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9859 - accuracy: 0.4752 - val_loss: 0.9809 - val_accuracy: 0.4730
Epoch 4/10
301/301 [==============================] - 6s 19ms/step - loss: 0.9789 - accuracy: 0.4904 - val_loss: 0.9690 - val_accuracy: 0.4842
Epoch 5/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9768 - accuracy: 0.4843 - val_loss: 0.9696 - val_accuracy: 0.5000
Epoch 6/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9756 - accuracy: 0.4878 - val_loss: 0.9627 - val_accuracy: 0.4800
Epoch 7/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9725 - accuracy: 0.4941 - val_loss: 0.9610 - val_accuracy: 0.5033

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

76/76 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0       0.45      0.47      0.46       921
           1       0.00      0.00      0.00       341
           2       0.56      0.71      0.62      1144

    accuracy                           0.52      2406
   macro avg       0.34      0.39      0.36      2406
weighted avg       0.44      0.52      0.47      2406



c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# 單一 3 層 CNN 和 LSTM 網絡

three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 6s 12ms/step - loss: 1.0029 - accuracy: 0.4535 - val_loss: 1.0131 - val_accuracy: 0.4027
Epoch 2/10
301/301 [==============================] - 3s 10ms/step - loss: 0.9910 - accuracy: 0.4751 - val_loss: 0.9811 - val_accuracy: 0.4904
Epoch 3/10
301/301 [==============================] - 3s 10ms/step - loss: 0.9793 - accuracy: 0.4847 - val_loss: 0.9674 - val_accuracy: 0.4917
Epoch 4/10
 84/301 [=======>......................] - ETA: 2s - loss: 0.9880 - accuracy: 0.4866

In [ ]:
# 多重 CNN 和 LSTM 網絡

multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 8s 19ms/step - loss: 0.9307 - accuracy: 0.5341 - val_loss: 0.8799 - val_accuracy: 0.5889
Epoch 2/10
301/301 [==============================] - 6s 19ms/step - loss: 0.8628 - accuracy: 0.5853 - val_loss: 0.8838 - val_accuracy: 0.5777
Epoch 3/10
301/301 [==============================] - 5s 18ms/step - loss: 0.8256 - accuracy: 0.6108 - val_loss: 0.8332 - val_accuracy: 0.6060
Epoch 4/10
301/301 [==============================] - 5s 18ms/step - loss: 0.7841 - accuracy: 0.6412 - val_loss: 0.8223 - val_accuracy: 0.6264
Epoch 5/10
301/301 [==============================] - 5s 17ms/step - loss: 0.7595 - accuracy: 0.6581 - val_loss: 0.8154 - val_accuracy: 0.6351
Epoch 6/10
301/301 [==============================] - 5s 18ms/step - loss: 0.7225 - accuracy: 0.6862 - val_loss: 0.8069 - val_accuracy: 0.6268
Epoch 7/10
301/301 [==============================] - 5s 18ms/step - loss: 0.6900 - accuracy: 0.6981 - val_loss: 0.8154 - val_accuracy: 0.6413

In [ ]:
from keras.layers import Bidirectional

# 單一 3 層 CNN 和雙向 LSTM 網絡

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 10s 19ms/step - loss: 0.9515 - accuracy: 0.5181 - val_loss: 0.8945 - val_accuracy: 0.5885
Epoch 2/10
301/301 [==============================] - 5s 17ms/step - loss: 0.8674 - accuracy: 0.5872 - val_loss: 0.8650 - val_accuracy: 0.5885
Epoch 3/10
301/301 [==============================] - 5s 16ms/step - loss: 0.8286 - accuracy: 0.6088 - val_loss: 0.8473 - val_accuracy: 0.5977
Epoch 4/10
301/301 [==============================] - 5s 17ms/step - loss: 0.7968 - accuracy: 0.6362 - val_loss: 0.8459 - val_accuracy: 0.6064
Epoch 5/10
301/301 [==============================] - 86s 287ms/step - loss: 0.7618 - accuracy: 0.6471 - val_loss: 0.8416 - val_accuracy: 0.6064
Epoch 6/10
301/301 [==============================] - 41s 135ms/step - loss: 0.7158 - accuracy: 0.6783 - val_loss: 0.8572 - val_accuracy: 0.6097
Epoch 7/10
301/301 [==============================] - 5s 17ms/step - loss: 0.6823 - accuracy: 0.6991 - val_loss: 0.8747 - val_accuracy: 0

In [ ]:
# 多重 CNN 和雙向 LSTM 網絡

multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 14s 34ms/step - loss: 0.9302 - accuracy: 0.5341 - val_loss: 0.8690 - val_accuracy: 0.5989
Epoch 2/10
301/301 [==============================] - 10s 34ms/step - loss: 0.8471 - accuracy: 0.5960 - val_loss: 0.8455 - val_accuracy: 0.6035
Epoch 3/10
301/301 [==============================] - 10s 33ms/step - loss: 0.8040 - accuracy: 0.6292 - val_loss: 0.8211 - val_accuracy: 0.6355
Epoch 4/10
301/301 [==============================] - 10s 32ms/step - loss: 0.7661 - accuracy: 0.6581 - val_loss: 0.8198 - val_accuracy: 0.6251
Epoch 5/10
301/301 [==============================] - 10s 34ms/step - loss: 0.7291 - accuracy: 0.6744 - val_loss: 0.8024 - val_accuracy: 0.6372
Epoch 6/10
301/301 [==============================] - 10s 33ms/step - loss: 0.6794 - accuracy: 0.7059 - val_loss: 0.8063 - val_accuracy: 0.6276
Epoch 7/10
301/301 [==============================] - 10s 32ms/step - loss: 0.6394 - accuracy: 0.7213 - val_loss: 0.8318 - val_accuracy: